In [1]:
pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 KB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.4/662.4 KB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.1/167.1 KB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.8/328.8 KB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
pip install numba==0.53

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 47.9 MB/s eta 0:00:00
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.37.0
    Uninstalling llvmlite-0.37.0:
      Successfully uninstalled llvmlite-0.37.0
  Attempting uninstall: numba
    Found existing installation: numba 0.54.1
    Uninstalling numba-0.54.1:
      Successfully uninstalled numba-0.54.1


In [3]:
pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from http.server import SimpleHTTPRequestHandler,HTTPServer
from urllib import request,error
import pandas as pd
from pycaret.clustering import *
import urllib.parse
import sys

In [5]:
badwords = ['sleep','drop','uid','select','waitfor','delay','system','union','order by','group by']
def ExtractFeatures(path):
  path = urllib.parse.unquote(path)
  badwords_count = 0
  single_q = path.count("'")
  double_q = path.count("\"")
  dashes = path.counnt("-")
  braces = path.count("(")
  spaces = path.count(" ")
  for words in badwords:
    badwords_count+=path.count(word)
  lst = [single_q,double_q,dashes,braces,spaces,badwords_count]
  print(lst)
  return pd.DataFrame([lst],columns = ['single_q','double_q','dashes','braces','spaces','badwords'])

Training the model

In [12]:
http = pd.read_csv('https://raw.githubusercontent.com/harishankar2001/ML-based-WAF/main/httplog_final.csv')
http = http.dropna()
clu1 = setup(data = http, normalize = True, numeric_features = ['single_q','double_q','dashes','braces','spaces','badwords'],ignore_features = ['method','path','body','class'])
dbscan = create_model('dbscan',num_clusters = 2)

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.5926,19.7383,1.3642,0,0,0


INFO:logs:create_model_container: 1
INFO:logs:master_model_container: 1
INFO:logs:display_container: 2
INFO:logs:DBSCAN(algorithm='auto', eps=0.5, leaf_size=30, metric='euclidean',
       metric_params=None, min_samples=5, n_jobs=-1, p=None)
INFO:logs:create_model() succesfully completed......................................


Creating http proxy

In [ ]:
class SimpleHTTPProxy(SimpleHTTPRequestHandler):
  proxy_routes = {}
  @classmethod
  def set_routes(cls,proxy_routes):
    cls.proxy_routes = proxy_routes
  def do_GET(self):
    parts = self.path.split("/")
    print(parts)
    live_data = ExtractFeatures(parts[3])
    result = predict_model(dbscan,data = live_data)
    print(result['cluster'][0])
    if result['cluster'][0]=="cluster 1":
      print("Intrusion detected")
    if len(parts) >= 2:
      self.proxy_request('http://'+parts[2]+'/')
    else:
      super().do_get()
    def proxy_request(self,url):
      try:
        response = request.urlopen(url)
      except error.HTTPError as e:
        print('err')
        self.send_response_only(e.code)
        self.end_headers()
        return 
      self.send_response_only(response.status)
      for name,value in response.header.items():
        self.send_header(name,value)
      self.end_headers()
      self.copyfile(response,self.wfile)
SimpleHTTPProxy.set_routes({'proxy_route':'http://demo.testfire.net/'})
with HTTPServer(('127.0.0.1',1234),SimpleHTTPProxy) as httpd:
  host, port = httpd.socket.getsockname()
  print(f'Listening on http://{host}:{port}')
  try:
    httpd.serve_forever()
  except KeyboardInterrupt:
    print("\nKeyboard interrupt received, exiting......")
    sys.exit(0)

  
      

Listening on http://127.0.0.1:1234
